# Face Recog and Biometric

## Installs

In [ ]:
# !pip install ngrok opencv-python-headless deepface mtcnn tf-keras

## Preproc

In [20]:
import cv2

In [21]:

def preprocess_image(image_path):
    image = cv2.imread(image_path)

    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply histogram equalization for better contrast
    enhanced = cv2.equalizeHist(gray)

    # Convert back to BGR (optional for color-based models)
    enhanced = cv2.cvtColor(enhanced, cv2.COLOR_GRAY2BGR)

    # Save the enhanced image
    cv2.imwrite("enhanced_" + image_path, enhanced)
    return "enhanced_" + image_path

## DeepFace

In [22]:
from deepface import DeepFace
from mtcnn import MTCNN

In [23]:
def extract_faces(image_path, verbose=False):
    """
    Detects and extracts all faces from an image.
    Returns a list of cropped face images.
    """
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Initialize MTCNN detector
    detector = MTCNN()
    detections = detector.detect_faces(image_rgb)

    faces = []
    for detection in detections:
        x, y, width, height = detection['box']
        # Crop the face
        face = image_rgb[y:y+height, x:x+width]
        faces.append(face)
    if verbose:
        print(f"Detected {len(faces)} face(s) in {image_path}")
    return faces

In [24]:
# def verify_faces(image1_path, image2_path):
#     """
#     Compares each detected face in one image to each detected face in another image.
#     """
#     # Extract faces from both images
#     faces1 = extract_faces(image1_path)
#     faces2 = extract_faces(image2_path)

#     if len(faces1) == 0:
#         print(f"No faces detected in {image1_path}")
#         return
#     if len(faces2) == 0:
#         print(f"No faces detected in {image2_path}")
#         return

#     print(f"Detected {len(faces1)} face(s) in {image1_path}")
#     print(f"Detected {len(faces2)} face(s) in {image2_path}")

#     # Compare each face in image1 with each face in image2
#     for i, face1 in enumerate(faces1):
#         for j, face2 in enumerate(faces2):
#             try:
#                 # Save temporary cropped face images for DeepFace
#                 face1_path = f"temp_face1_{i}.jpg"
#                 face2_path = f"temp_face2_{j}.jpg"
#                 cv2.imwrite(face1_path, cv2.cvtColor(face1, cv2.COLOR_RGB2BGR))
#                 cv2.imwrite(face2_path, cv2.cvtColor(face2, cv2.COLOR_RGB2BGR))

#                 # Perform verification
#                 result = DeepFace.verify(face1_path, face2_path, detector_backend="mtcnn", enforce_detection=False)
#                 print(f"Comparison between Face {i+1} in Image1 and Face {j+1} in Image2: {result}")
#             except Exception as e:
#                 print(f"Error comparing Face {i+1} in Image1 and Face {j+1} in Image2: {e}")

In [25]:
def verify_against_allowed(allowed_images, unknown_images, verbose=False):
    """
    Verifies if any unknown face matches any face from the allowed list.

    Args:
        allowed_images (list): List of file paths for images of allowed people.
        unknown_images (list): List of file paths for images of unknown people.
        verbose (bool): If True, prints detailed logs.

    Returns:
        bool: True if any match is found, False otherwise.
    """
    # Extract all faces from allowed images
    allowed_faces = []
    for image_path in allowed_images:
        faces = extract_faces(image_path, verbose=verbose)
        allowed_faces.extend(faces)

    if len(allowed_faces) == 0:
        if verbose:
            print("No faces detected in allowed images.")
        return False

    if verbose:
        print(f"Total detected faces in allowed images: {len(allowed_faces)}")

    # Extract all faces from unknown images
    unknown_faces = []
    for image_path in unknown_images:
        faces = extract_faces(image_path, verbose=verbose)
        unknown_faces.extend(faces)

    if len(unknown_faces) == 0:
        if verbose:
            print("No faces detected in unknown images.")
        return False

    if verbose:
        print(f"Total detected faces in unknown images: {len(unknown_faces)}")

    # Compare each unknown face against each allowed face
    for i, unknown_face in enumerate(unknown_faces):
        for j, allowed_face in enumerate(allowed_faces):
            try:
                # Save temporary cropped face images for DeepFace
                unknown_face_path = f"temp_unknown_face_{i}.jpg"
                allowed_face_path = f"temp_allowed_face_{j}.jpg"
                cv2.imwrite(unknown_face_path, cv2.cvtColor(unknown_face, cv2.COLOR_RGB2BGR))
                cv2.imwrite(allowed_face_path, cv2.cvtColor(allowed_face, cv2.COLOR_RGB2BGR))

                # Perform verification
                result = DeepFace.verify(unknown_face_path, allowed_face_path, detector_backend="mtcnn", enforce_detection=False)
                if verbose:
                    print(f"Comparison between Unknown Face {i+1} and Allowed Face {j+1}: {result}")
                if result['verified']:
                    if verbose:
                        print(f"Match found: Unknown Face {i+1} matches Allowed Face {j+1}")
                    return True
            except Exception as e:
                if verbose:
                    print(f"Error comparing Unknown Face {i+1} and Allowed Face {j+1}: {e}")

    if verbose:
        print("No match found between unknown and allowed faces.")
    return False

In [57]:
# # Def jpgs
# jpg1 = 'dark' + '.jpg'
# jpg2 = 'musk' + '.jpg'

# # Enhance images
# image1_path = preprocess_image(jpg1)
# image2_path = preprocess_image(jpg2)

# Verify if two images belong to the same person
# verify_faces(jpg1, jpg2)

allowed_images = ["low2_close.jpg"] #, "musk_far.jpg", "musk_tilt.jpg"]  # Paths to allowed images
unknown_images = ["low1_far_tilt.jpg"]  # Paths to unknown images "low1_close.jpg", "low2_close.jpg", "low1_dark.jpg",

result = verify_against_allowed(allowed_images, unknown_images, verbose=True)
print("Verification Result:", result)

# Analyze a single image
analysis = DeepFace.analyze("low1_far_tilt.jpg", detector_backend="mtcnn")
print("Analysis: ", analysis)  # Includes demographic predictions like age, gender, emotion


Detected 2 face(s) in low2_close.jpg
Total detected faces in allowed images: 2
Detected 1 face(s) in low1_far_tilt.jpg
Total detected faces in unknown images: 1
Comparison between Unknown Face 1 and Allowed Face 1: {'verified': True, 'distance': 0.3781767691793787, 'threshold': 0.68, 'model': 'VGG-Face', 'detector_backend': 'mtcnn', 'similarity_metric': 'cosine', 'facial_areas': {'img1': {'x': 0, 'y': 0, 'w': 32, 'h': 45, 'left_eye': None, 'right_eye': None}, 'img2': {'x': 1, 'y': 0, 'w': 37, 'h': 51, 'left_eye': (29, 20), 'right_eye': (10, 19)}}, 'time': 4.71}
Match found: Unknown Face 1 matches Allowed Face 1
Verification Result: True


Action: race: 100%|██████████| 4/4 [00:10<00:00,  2.64s/it]  

Analysis:  [{'emotion': {'angry': 0.007898546027718112, 'disgust': 1.2873186880036155e-14, 'fear': 4.621165618300438, 'happy': 0.0002979694500027108, 'sad': 5.572805553674698, 'surprise': 0.003523028499330394, 'neutral': 89.79431390762329}, 'dominant_emotion': 'neutral', 'region': {'x': 163, 'y': 138, 'w': 34, 'h': 45, 'left_eye': (194, 155), 'right_eye': (182, 155)}, 'face_confidence': 0.96, 'age': 28, 'gender': {'Woman': 12.587174773216248, 'Man': 87.41282224655151}, 'dominant_gender': 'Man', 'race': {'asian': 10.009870381339834, 'indian': 11.383242772012087, 'black': 13.554257280141982, 'white': 34.205703044617536, 'middle eastern': 17.33785031293038, 'latino hispanic': 13.509081424365213}, 'dominant_race': 'white'}]


## Deploy

In [27]:
# !pip install flask

In [62]:
# !pip install ngrok
# !ngrok http 5000

In [61]:
import tempfile
import os
import json
from flask import Flask, request, jsonify
from deepface import DeepFace
import numpy as np

app = Flask(__name__)

def convert_numpy_types(obj):
    """Recursively convert NumPy types to native Python types."""
    if isinstance(obj, np.integer):  # Handles np.int32, np.int64, etc.
        return int(obj)
    elif isinstance(obj, np.floating):  # Handles np.float32, np.float64, etc.
        return float(obj)
    elif isinstance(obj, np.ndarray):  # Converts np.ndarray to a Python list
        return obj.tolist()
    elif isinstance(obj, dict):  # Recursively convert dict values
        return {k: convert_numpy_types(v) for k, v in obj.items()}
    elif isinstance(obj, list):  # Recursively convert list elements
        return [convert_numpy_types(x) for x in obj]
    else:
        return obj  # If it's a normal object, return it as is

@app.route('/analyze', methods=['POST'])
def analyze_face():
    try:
        # Check if an image file is present in the request
        if 'image' not in request.files:
            return jsonify({"error": "No image file provided"}), 400

        # Retrieve the image file from the request
        image_file = request.files['image']

        # Save the image to a temporary file
        temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.jpg')
        image_file.save(temp_file.name)
        
        # Close the temporary file to release the lock
        temp_file.close()

        # Perform face analysis using DeepFace
        analysis = DeepFace.analyze(temp_file.name, detector_backend="mtcnn")
        print("image analyzed")

        # Convert NumPy types using the custom function
        clean_analysis = convert_numpy_types(analysis)
        
        # Try serializing with json.dumps to catch errors
        json_data = json.dumps({"analysis": clean_analysis}, default=str)  # Use str for non-serializable types
        print(json_data)

        # Delete the temporary file after use
        os.remove(temp_file.name)
        
        return json_data
    
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route('/verify', methods=['POST'])
def verify_faces():
    try:
        # Check if image files are present in the request
        if 'unknown_images' not in request.files: # 'allowed_images' not in request.files or 
            return jsonify({"error": "Unknown image files must be provided"}), 400

        # allowed_files = request.files.getlist('allowed_images')
        allowed_files = allowed_images
        unknown_files = request.files.getlist('unknown_images')
        verbose = request.form.get('verbose', 'false').lower() == 'true'

        allowed_image_paths = allowed_files
        unknown_image_paths = []

        # # Save allowed images as temporary files
        # for image_file in allowed_files:
        #     temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.jpg')
        #     image_file.save(temp_file.name)
        #     temp_file.close()
        #     allowed_image_paths.append(temp_file.name)

        # Save unknown images as temporary files
        for image_file in unknown_files:
            temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.jpg')
            image_file.save(temp_file.name)
            temp_file.close()
            unknown_image_paths.append(temp_file.name)

        # Perform verification using DeepFace
        result = verify_against_allowed(allowed_image_paths, unknown_image_paths, verbose)

        # Clean up temporary files
        for path in unknown_image_paths: # allowed_image_paths + 
            try:
                os.remove(path)
            except Exception as e:
                print(f"Failed to delete temp file {path}: {e}")
        
        return jsonify({"verification_result": result})

    except Exception as e:
        return jsonify({"error": str(e)}), 500

# @app.route('/print', methods=['POST'])
# def print_message():
#     try:
#         # Read raw data from the request body
#         message = request.data.decode('utf-8').strip()

#         # Validate the message
#         if not message:
#             return jsonify({"error": "Empty message provided"}), 400

#         # Print the message to the terminal
#         print(f"Received message: {message}")

#         # Return a success response
#         return jsonify({"status": "Message printed successfully"}), 200

#     except Exception as e:
#         return jsonify({"error": str(e)}), 500

# @app.route('/upload', methods=['POST'])
# def upload_binary():
#     try:
#         # Read binary data from the request body
#         binary_data = request.data

#         # Validate the data
#         if not binary_data:
#             return jsonify({"error": "Empty binary data provided"}), 400

#         # Save the binary data to a file
#         file_path = "uploaded_image.jpg"
#         with open(file_path, "wb") as f:
#             f.write(binary_data)

#         print(f"Binary data saved to {file_path}")

#         # Return a success response
#         return jsonify({"status": "Binary data received and saved successfully"}), 200

#     except Exception as e:
#         return jsonify({"error": str(e)}), 500

@app.route('/', methods=['GET'])
def health_check():
    return jsonify({"status": "OK"})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://10.7.184.38:5000
Press CTRL+C to quit
10.7.184.38 - - [21/Dec/2024 17:04:21] "POST /upload HTTP/1.1" 200 -


Binary data saved to uploaded_image.jpg


## Helpers

In [ ]:
import os
import zipfile

# Directory containing the images
image_directory = '/content'  # Update this to the correct path

# Create a zip file and add only .jpg files
with zipfile.ZipFile('/content/jpg_images.zip', 'w') as zipf:
    for root, dirs, files in os.walk(image_directory):
        for file in files:
            if file.endswith('.jpg'):
                full_path = os.path.join(root, file)
                zipf.write(full_path, os.path.relpath(full_path, image_directory))


In [ ]:
from google.colab import files

# Download the zip file containing only .jpg images
files.download('/content/jpg_images.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>